In [1]:
import torch

x = torch.rand(5, 3)
print(x)

tensor([[0.3583, 0.0042, 0.6256],
        [0.4306, 0.0392, 0.5870],
        [0.9865, 0.8396, 0.4966],
        [0.0627, 0.7900, 0.7678],
        [0.3278, 0.2250, 0.3016]])


In [12]:
torch.backends.mps.is_available()
torch.backends.mps.is_built()
mps_device = torch.device("mps")
x = torch.ones(45, device="mps")

In [14]:
ans = x * 45
ans

tensor([45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45.,
        45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45.,
        45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45.,
        45., 45., 45.], device='mps:0')